In [2]:
%run "/home/jovyan/work/workspace/functions/create_spark_session.ipynb"

In [3]:
%run "/home/jovyan/work/workspace/functions/paths.ipynb"

In [4]:
%run "/home/jovyan/work/workspace/functions/dates.ipynb"

In [5]:
!pip install -q --upgrade ipywidgets\
runcell

In [6]:
import pygwalker as pyg

In [7]:
f'{gold}/sales'

'/home/jovyan/work/lake/gold//sales'

In [8]:
fact_sales = spark.read.format("delta").load(f"{gold}sales")

df_viz = fact_sales.toPandas()

walker = pyg.walk(df_viz)

Box(children=(HTML(value='\n<div id="ifr-pyg-00064ae3aa25172dtSupybT8JlgNHw0Q" style="height: auto">\n    <hea…

In [9]:
!ls /home/jovyan/work/lake/gold

channel_group  sales  sales_fact  sales_location  sales_product  sales_time


In [10]:
sales = spark.read.format('delta').load(f'{gold}sales')

In [11]:
sales_location = spark.read.format('delta').load(f'{gold}sales_location')

In [12]:
sales_product = spark.read.format('delta').load(f'{gold}sales_product')

In [13]:
sales_time = spark.read.format('delta').load(f'{gold}sales_time')

In [14]:
channel_group = spark.read.format('delta').load(f'{gold}channel_group')

In [15]:
sales_fact = spark.read.format('delta').load(f'{gold}sales_fact')

In [16]:
sales.createOrReplaceTempView('sales')
sales_product.createOrReplaceTempView('sales_product')
sales_time.createOrReplaceTempView('sales_time')
channel_group.createOrReplaceTempView('channel_group')
sales_fact.createOrReplaceTempView('sales_fact')

#### Create queries/scripts that can answer following business questions using the data structure created:
Expected result: Data processes and results.

What are the Top 3 Trade Groups (TRADE_GROUP_DESC) for each Region (Btlr_Org_LVL_C_Desc) in sales ($ Volume)?


In [17]:
sales.columns

['dtregister',
 'cdbrand',
 'dsbrand',
 'dsregion',
 'cdgroup',
 'tpgroup',
 'dsgroup',
 'dschannel',
 'dspack',
 'dscatpack',
 'dsnmpack',
 'qtvolume',
 'dsyear',
 'dsmonth',
 'dsquarter',
 'dtload']

In [30]:
top3 = spark.sql('''
WITH sales_per_region as (SELECT 
        c.tpgroup, 
        s.dsregion, 
        SUM(s.qtvolume) as ttlvolume,
        RANK() OVER (PARTITION BY s.dsregion ORDER BY SUM(s.qtvolume) DESC) as ranking
    FROM sales s 
    LEFT JOIN channel_group c ON c.cdgroup = s.cdgroup
    GROUP BY c.tpgroup, s.dsregion
)

select spr.* from sales_per_region spr where spr.ranking <=3
''')

In [31]:
top3.toPandas()

,tpgroup,dsregion,ttlvolume,ranking
0,GROCERY,CANADA,2845223.55,1
1,ACADEMIC,CANADA,289981.68,2
2,SERVICES,CANADA,265696.62,3
3,GROCERY,GREAT LAKES,6298686.16,1
4,ACADEMIC,GREAT LAKES,761313.06,2
5,SERVICES,GREAT LAKES,530709.27,3
6,GROCERY,MIDWEST,5288908.95,1
7,ACADEMIC,MIDWEST,462949.92,2
8,SERVICES,MIDWEST,370923.70,3
9,GROCERY,NORTHEAST,5964821.33,1


How much sales ($ Volume) each brand (BRAND_NM) achieved per month?


In [32]:
sales.toPandas()

,dtregister,cdbrand,dsbrand,dsregion,cdgroup,tpgroup,dsgroup,dschannel,dspack,dscatpack,dsnmpack,qtvolume,dsyear,dsmonth,dsquarter,dtload
0,2006-01-01,3440,LEMON,NORTHEAST,0,GOV & MILITARY,MILITARY/GOVT,GOVERNMENT(NON-MILITARY),N20O,20Z/600ML,20Z NRP 24L,95.00,2006,1,1,2026-02-15
1,2006-01-01,3440,LEMON,SOUTHEAST,4,GROCERY,CONVENIENCE RETAIL,CONVENIENCE STORE,N20O,20Z/600ML,20Z NRP 24L,1342.50,2006,1,1,2026-02-15
2,2006-01-01,3440,LEMON,WEST,1,SERVICES,WORKPLACE,PLANT / OFFICE,N20O,20Z/600ML,20Z NRP 24L,156.45,2006,1,1,2026-02-15
3,2006-01-08,3440,LEMON,CANADA,3,OTHER,OTHER SMALL STORES,ALL OTHER,N20O,20Z/600ML,.591L NRP 24L,0.00,2006,1,2,2026-02-15
4,2006-01-08,3441,RASPBERRY,MIDWEST,4,GROCERY,CONVENIENCE RETAIL,CONVENIENCE STORE,N20O,20Z/600ML,20Z NRP 24L,2290.00,2006,1,2,2026-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16146,2006-03-29,3440,LEMON,CANADA,4,GROCERY,DRUG STORES,DRUG STORE,N20O,20Z/600ML,.591L NRP 24L *,219.83,2006,3,5,2026-02-15
16147,2006-03-29,3441,RASPBERRY,SOUTHWEST,4,GROCERY,RETAIL COLD,AUTOMOTIVE SERVICES,N20O,20Z/600ML,20z NRP 24L S,12.50,2006,3,5,2026-02-15
16148,2006-03-29,3554,STRAWBERRY,CANADA,3,OTHER,OTHER SMALL STORES,MILITARY-EXCHANGE,N20O,20Z/600ML,.591L NRP 24L,2.49,2006,3,5,2026-02-15
16149,2006-03-29,3440,LEMON,SOUTHEAST,1,SERVICES,TRANSPORTATION,TRANSPORTATION,N20O,20Z/600ML,20Z NRP 24L,6.25,2006,3,5,2026-02-15


In [43]:
sales_volume = spark.sql(

'''
SELECT 
s.dsbrand, s.dsmonth, sum(s.qtvolume) as ttlvolume

FROM sales s 
    GROUP BY  s.dsbrand, s.dsmonth

'''
)

In [44]:
sales_volume.toPandas()

,dsbrand,dsmonth,ttlvolume
0,GRAPE,3,7.50
1,RASPBERRY,1,392551.70
2,RASPBERRY,2,409661.52
3,STRAWBERRY,1,315902.65
4,LEMON,3,765942.58
5,LEMON,2,550940.85
6,RASPBERRY,3,587522.56
7,STRAWBERRY,2,345518.40
8,STRAWBERRY,3,475032.18
9,LEMON,1,505845.22


Which are the lowest brand (BRAND_NM) in sales ($ Volume) for each region (Btlr_Org_LVL_C_Desc)?

In [77]:
lowest_sales_general = spark.sql('''
SELECT 
s.dsbrand, sum(s.qtvolume) as ttlvolume, s.dsregion, 
RANK() OVER (PARTITION BY s.dsbrand, s.dsregion ORDER BY  sum(s.qtvolume) DESC) as ranking

FROM sales s
    GROUP BY s.dsbrand, s.dsregion
    ORDER BY s.dsregion ASC, ranking desc
''')

lowest_sales_general.toPandas()

,dsbrand,ttlvolume,dsregion,ranking
0,LEMON,139041.68,CANADA,1
1,RASPBERRY,132547.52,CANADA,1
2,STRAWBERRY,74009.61,CANADA,1
3,LEMON,299199.90,GREAT LAKES,1
4,RASPBERRY,234598.87,GREAT LAKES,1
5,STRAWBERRY,208747.90,GREAT LAKES,1
6,LEMON,240295.42,MIDWEST,1
7,RASPBERRY,213987.51,MIDWEST,1
8,STRAWBERRY,149206.78,MIDWEST,1
9,LEMON,337429.32,NORTHEAST,1


In [88]:
lowest_volume = spark.sql(
'''
WITH lowest_volume as (
SELECT 
s.dsbrand, sum(s.qtvolume) as ttlvolume, s.dsregion,
RANK() OVER (PARTITION BY s.dsregion ORDER BY sum(s.qtvolume) ASC) as ranking
FROM sales s
    GROUP BY s.dsbrand, s.dsregion)

SELECT l.dsbrand,l.ttlvolume as minvolume, l.dsregion FROM lowest_volume l
  WHERE l.ranking = 1   
    ORDER BY l.dsregion asc
''')


In [89]:
lowest_volume.toPandas()

,dsbrand,minvolume,dsregion
0,STRAWBERRY,74009.61,CANADA
1,STRAWBERRY,208747.90,GREAT LAKES
2,STRAWBERRY,149206.78,MIDWEST
3,STRAWBERRY,195581.49,NORTHEAST
4,RASPBERRY,223958.50,SOUTHEAST
5,GRAPE,7.50,SOUTHWEST
6,STRAWBERRY,134511.18,WEST


In [91]:
top3.coalesce(1).write.format('csv').save(workdir+'export/top3')

In [92]:
sales_volume.coalesce(1).write.format('csv').save(workdir+'export/sales_volume')

In [ ]:
top3.coalesce(1).write.format('csv').save(workdir+'export/top3')